In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/bodyperformance'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bodyperformance/bodyPerformance.csv


In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.16" 2022-07-19; OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0ovxhtpz
  JVM stdout: /tmp/tmp0ovxhtpz/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp0ovxhtpz/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.1
H2O_cluster_version_age:,2 months and 4 days
H2O_cluster_name:,H2O_from_python_unknownUser_9gwja1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
from h2o.automl import H2OAutoML

In [3]:
df = h2o.import_file('/kaggle/input/bodyperformance/bodyPerformance.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
df.head()

age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
27,M,172.3,75.24,21.3,80,130,54.9,18.4,60,217,C
25,M,165,55.8,15.7,77,126,36.4,16.3,53,229,A
31,M,179.6,78,20.1,92,152,44.8,12,49,181,C
32,M,174.5,71.1,18.4,76,147,41.4,15.2,53,219,B
28,M,173.8,67.7,17.1,70,127,43.5,27.1,45,217,B
36,F,165.4,55.4,22,64,119,23.8,21,27,153,B
42,F,164.5,63.7,32.2,72,135,22.7,0.8,18,146,D
33,M,174.9,77.2,36.9,84,137,45.9,12.3,42,234,B
54,M,166.8,67.5,27.6,85,165,40.4,18.6,34,148,C
28,M,185,84.6,14.4,81,156,57.9,12.1,55,213,B


In [11]:
df['class'].unique()

C1
A
B
C
D


In [13]:
y ='class'
X = df.columns.remove(y)

In [14]:
splits = df.split_frame(ratios=[0.75] , seed=1)
train = splits[0]
test = splits[1]

In [15]:
aml = H2OAutoML(max_runtime_secs=120)
aml.train(x=X , y=y , training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_1_20221124_20042

No summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.10141404365782167
RMSE: 0.31845571694950253
LogLoss: 0.32832087923035347
Null degrees of freedom: 9983
Residual degrees of freedom: 9923
Null deviance: 27680.222861930604
Residual deviance: 6555.911316471698
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
A     B     C     D     Error      Rate
----  ----  ----  ----  ---------  -----------
2495  34    10    1     0.0177165  45 / 2,540
316   2122  43    16    0.15018    375 / 2,497
88    200   2193  6     0.118215   294 / 2,487
19    50    139   2252  0.0845528  208 / 2,460
2918  2406  2385  2275  0.0923478  922 / 9,984

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.907652
2    0.979567
3    0.997095
4    1

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.21041538175017555
RMSE: 0.4587105642452281
LogLoss: 0.6252220867415932
Null degrees of freedom: 10073
Residual degrees of freedom: 10010
Null deviance: 27938.23339520976
Residual deviance: 12596.97460366962
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
A     B     C     D     Error     Rate
----  ----  ----  ----  --------  --------------
2203  325   31    2     0.139789  358 / 2,561
592   1526  343   61    0.394925  996 / 2,522
197   497   1699  121   0.324185  815 / 2,514
33    118   301   2025  0.182479  452 / 2,477
3025  2466  2374  2209  0.260175  2,621 / 10,074

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.739825
2    0.935974
3    0.989776
4    1

Cross-Validation Metrics Summary: 
                         mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.739743  0.00788532  0.75305       0.731943      0.737796      0.737389      0.738537
auc                      nan       0           nan           nan           nan           nan           nan
err                      0.260257  0.00788532  0.24695       0.268057      0.262204      0.262611      0.261463
err_count                524.2     11.5629     506           527           521           531           536
logloss                  0.62516   0.0113636   0.63129       0.630206      0.614285      0.612064      0.637956
max_per_class_error      0.396029  0.0247262   0.373214      0.437229      0.387234      0.384016      0.398453
mean_per_class_accuracy  0.739268  0.0106494   0.757331      0.729287      0.734579      0.737524      0.737621
mean_per_class_error     0.260732  0.0106494   0.242669      0.270713      0.265421      0.262476      0.262379
mse                      0.210408  0.00249854  0.210929      0.212548      0.207346      0.2083        0.212918
null_deviance            5587.65   103.198     5683.93       5453.28       5511.59       5606.15       5683.28
pr_auc                   nan       0           nan           nan           nan           nan           nan
r2                       0.831544  0.00484458  0.824466      0.831299      0.837946      0.833078    

In [16]:
lb = aml.leaderboard
lb.head()

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20221124_20042,0.260344,0.625222,0.458711,0.210415
StackedEnsemble_BestOfFamily_1_AutoML_1_20221124_20042,0.260365,0.634222,0.461372,0.212864
StackedEnsemble_BestOfFamily_2_AutoML_1_20221124_20042,0.263606,0.636037,0.462997,0.214367
GBM_3_AutoML_1_20221124_20042,0.267215,0.677425,0.486664,0.236842
GBM_1_AutoML_1_20221124_20042,0.268254,0.644293,0.467531,0.218585
GBM_2_AutoML_1_20221124_20042,0.269546,0.681044,0.488802,0.238928
GBM_4_AutoML_1_20221124_20042,0.271609,0.689204,0.489526,0.239635
XGBoost_2_AutoML_1_20221124_20042,0.272795,0.668633,0.474776,0.225413
XGBoost_1_AutoML_1_20221124_20042,0.2761,0.676686,0.472726,0.22347
DRF_1_AutoML_1_20221124_20042,0.299422,1.26669,0.504842,0.254865


In [17]:
pre = aml.leader.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [18]:
pre.head()

predict,A,B,C,D
B,0.00314219,0.486008,0.478867,0.0319834
D,2.91668e-10,5.36449e-06,0.000155613,0.999839
B,0.0163953,0.734215,0.210578,0.0388117
C,1.22697e-05,0.0159261,0.866627,0.117434
C,3.80059e-05,0.0181135,0.933457,0.0483911
B,0.292362,0.479227,0.213327,0.0150835
D,1.8595e-07,0.000369723,0.00206524,0.997565
A,0.748594,0.212218,0.0334846,0.00570297
A,0.520101,0.275642,0.149734,0.0545229
B,0.120355,0.747381,0.113043,0.0192215


In [19]:
aml.leader.model_performance(test)

ModelMetricsMultinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.20328297948959065
RMSE: 0.4508691378766023
LogLoss: 0.6136127005440389
Null degrees of freedom: 3318
Residual degrees of freedom: 3258
Null deviance: 9205.5544415622
Residual deviance: 4073.1611062113307
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
A    B    C    D    Error     Rate
---  ---  ---  ---  --------  -----------
684  94   7    2    0.130877  103 / 787
170  519  113  23   0.370909  306 / 825
65   148  589  33   0.294611  246 / 835
13   43   95   721  0.173165  151 / 872
932  804  804  779  0.242844  806 / 3,319

Top-4 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.757156
2    0.931606
3    0.988852
4    1